In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from neurovlm.retrieval_resources import (
    _load_dataframe, _load_latent_text
)
from neurovlm.data import data_dir
from neurovlm.train import Trainer, which_device
from neurovlm.models import ConceptClf

# Concept Classifier

The concept classifier predicts which concepts are present given a latent neuro embeddings. The top-10 related concepts are passed to an LLM to summarize the brain map. Here, Llama-3.1-8B-Instruct is used to generated interpretations. Any language model may be used. Larger models or models trained one neuroscience literature may provided better brain map interpretations.

In [2]:
# N-gram embeddings, from 06_n_grams.ipynb
ngram_emb = torch.load(data_dir / "ngram_emb.pt")

# load text
df = _load_dataframe()
df.sort_values(by="pmid", inplace=True)
text = df["name"] + " [SEP] " + df["description"]

# load pre-computed ngrams from 06_n_grams.ipynb
X = np.load(data_dir / "ngram_matrix.npy")
features = np.load(data_dir / "ngram_labels.npy")

# load latent text
latent, pmids = _load_latent_text()

In [3]:
# cosine similarity as target
y = latent @ (ngram_emb / ngram_emb.norm(dim=1)[:, None]).T
m = (y < 0.) | (torch.from_numpy(X) == 0.)
y[m==1] = 0.

# transform cosine similarity ~= probabilities
t = 0.03
tau = 0.08
y = torch.sigmoid((y - t)/ tau)

y[m] = 0.
y = y.numpy()

In [15]:
import torch.nn.functional as F
proj_head = torch.load(data_dir / f"proj_head_image_infonce.pt", weights_only=False, map_location="cpu")

In [19]:
# ensure latent neuro vectors align with df
latent_neuro, pmid = torch.load(
    data_dir / "latent_neuro.pt", weights_only=False, map_location="cpu"
).values()
with torch.no_grad():
    latent_neuro = F.normalize(proj_head(F.normalize(latent_neuro, dim=1)), dim=1)

assert (df["pmid"] == df["pmid"].sort_values()).all()

mask = df['pmid'].isin(pmid)
df, y = df[mask], y[mask]
df.reset_index(inplace=True, drop=True)

In [20]:
# load data splits
train_ids, test_ids, val_ids = torch.load(data_dir / "pmids_split.pt", weights_only=False).values()
train_ids.sort()
val_ids.sort()
test_ids.sort()

def split(df, latent, y, pmids, device):
    mask = df['pmid'].isin(pmids).to_numpy()
    X = latent[torch.from_numpy(mask)].clone().to(device)
    y = torch.from_numpy(y[mask].copy()).float().to(device)
    pmids = pmids[pd.Series(pmids).isin(df["pmid"])]
    return X, y, pmids

device = which_device()
X_train, y_train, train_ids = split(df, latent_neuro, y, train_ids, device)
X_val, y_val, val_ids = split(df, latent_neuro, y,  val_ids, device)
X_test, y_test, test_ids = split(df, latent_neuro, y, test_ids, device)

# ensure sorted
assert (df['pmid'] == df['pmid'].sort_values()).all()
assert (train_ids == np.sort(train_ids)).all()
assert (val_ids == np.sort(val_ids)).all()
assert (test_ids == np.sort(test_ids)).all()

In [22]:
clf = ConceptClf(X.shape[1]).to(device)

loss_fn = nn.BCEWithLogitsLoss()

trainer = Trainer(
    clf,
    loss_fn,
    lr=3e-5,
    n_epochs=201,
    batch_size=1028,
    optimizer=torch.optim.AdamW,
    X_val=X_val,
    y_val=y_val,
    interval=20,
    device=device
)

trainer.fit(X_train, y_train)

trainer.save(data_dir / "concept_clf.pt")

Epoch: -1, val loss: 0.6932
Epoch: 0, val loss: 0.68414
Epoch: 20, val loss: 0.042197
Epoch: 40, val loss: 0.041974
Epoch: 60, val loss: 0.041963
Epoch: 80, val loss: 0.041961
Epoch: 100, val loss: 0.041961
Epoch: 120, val loss: 0.041959
Epoch: 140, val loss: 0.04196
Epoch: 160, val loss: 0.041957
Epoch: 180, val loss: 0.041957
Epoch: 200, val loss: 0.041956


In [23]:
from neurovlm.retrieval_resources import _load_masker, _load_autoencoder
import gzip, pickle
import nibabel as nib
from nilearn.image import resample_to_img

masker = _load_masker()
autoencoder = _load_autoencoder()

# Load network atlases
with gzip.open(data_dir / "networks_arrays.pkl.gz", "rb") as f:
    networks = pickle.load(f)
    
networks = [(k, n, nib.Nifti1Image(networks[k][n]["array"], affine=networks[k][n]["affine"]))
            for k in networks.keys() for n in networks[k].keys()]

In [24]:
i = 0

x = masker.transform(
    resample_to_img(networks[i][2], masker.mask_img, interpolation="nearest", force_resample=True, copy_header=True)
)
x = autoencoder.encoder(torch.from_numpy(x))

scores = torch.sigmoid(clf(x.to("cuda")).cpu().detach())
print(f"top predicted terms for {networks[i][0] + "_" + networks[i][1]}:")
features[scores.argsort(descending=True)][:50]

top predicted terms for Du_VIS-P:


array(['visual', 'prefrontal cortex pfc', 'frontal gyrus ifg', 'motor',
       'memory', 'insula', 'age', 'cingulate cortex left',
       'bilateral anterior cingulate', 'relative healthy controls',
       'inferior frontal gyri', 'temporal gyrus mtg',
       'superior temporal cortex', 'inferior frontal gyrus',
       'anterior cingulate cortex', 'gyrus inferior parietal',
       'precuneus', 'self', 'dorsal', 'gyrus right inferior', 'social',
       'right dorsolateral prefrontal', 'gray matter density', 'early',
       'left inferior temporal', 'posterior parietal cortex',
       'precuneus posterior cingulate', 'ventral medial prefrontal',
       'inferior middle frontal', 'superior parietal lobe',
       'medial superior frontal', 'default mode network',
       'cognitive control network', 'superior middle temporal',
       'dorsolateral prefrontal cortex', 'working memory',
       'orbital frontal cortex', 'parietal lobule left',
       'middle frontal gyrus', 'limbic', 'medial p